규호버전/모델26개 만들기

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import warnings
import datetime
warnings.filterwarnings("ignore")
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, Dropout, MaxPool2D, Activation,
    Flatten, Dense, Input, Concatenate, LeakyReLU, Add, AveragePooling2D
)
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
# from efficientnet.keras import EfficientNetB0

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import LearningRateScheduler
from tqdm.notebook import tqdm
import pandas as pd

In [37]:
train = pd.read_csv('/content/drive/My Drive/데이콘 팀플/데이콘 공유/train.csv')
test = pd.read_csv('/content/drive/My Drive/데이콘 팀플/데이콘 공유/test.csv')

x_train = train.drop(['id', 'digit', 'letter'], axis=1).values
x_train = x_train.reshape(-1, 28, 28, 1)
x_train = x_train/255

y = train['digit']
y_train = np.zeros((len(y), len(y.unique())))

for i, digit in enumerate(y):
    y_train[i, digit] = 1
x_test = test.drop(['id', 'letter'], axis=1).values
x_test = x_test.reshape(-1, 28, 28, 1)
x_test = x_test/255
x_test_letter = test['letter']

z_train = train['letter']
z_test = test['letter']

In [38]:
len(train),len(test)

(2048, 20480)

In [39]:
import string 

alphabet = list(string.ascii_uppercase)
alphabet.index('D')

3

In [40]:
X_train=[]
Y_train=[]
for alpha in alphabet:
  x_list = []
  y_list = []
  X_train.append(x_list)
  Y_train.append(y_list)
  for x, y, z in zip(x_train, y_train, z_train):
    if alpha == z:
      x_list.append(x)
      y_list.append(y)

In [41]:
len(X_train), len(Y_train)

(26, 26)

In [42]:
X_test = []
for alpha in alphabet:
  x_list = []
  X_test.append(x_list)
  for x, z  in zip(x_test, z_test):
    if alpha == z:
      x_list.append(x)

In [43]:
len(X_test)

26

In [44]:
for x_trn in X_train:
  print(len(x_trn))

72
71
74
76
66
80
87
89
64
67
90
89
86
77
90
66
82
81
76
82
85
73
69
79
86
91


In [45]:
np.array(X_train[0])

array([[[[0.01176471],
         [0.        ],
         [0.00784314],
         ...,
         [0.01176471],
         [0.01568627],
         [0.00392157]],

        [[0.        ],
         [0.01176471],
         [0.        ],
         ...,
         [0.01568627],
         [0.00784314],
         [0.01176471]],

        [[0.01568627],
         [0.        ],
         [0.01176471],
         ...,
         [0.00392157],
         [0.00784314],
         [0.00392157]],

        ...,

        [[0.00392157],
         [0.01568627],
         [0.01176471],
         ...,
         [0.        ],
         [0.00784314],
         [0.        ]],

        [[0.        ],
         [0.01176471],
         [0.00784314],
         ...,
         [0.00784314],
         [0.00392157],
         [0.        ]],

        [[0.01568627],
         [0.00784314],
         [0.00784314],
         ...,
         [0.01176471],
         [0.00392157],
         [0.00784314]]],


       [[[0.00784314],
         [0.01176471],
         [0.  

Train model

In [46]:
# data 뻥튀기

datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range = 0.01, # Randomly zoom image 
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,
    fill_mode="nearest"
        )



모델 만들기 (A~Z)

In [47]:
def basic_cnn_model():
    # dropout_rate=0.5
    model = Sequential()
    model.add(Conv2D(256, kernel_size = 3, kernel_initializer='he_normal', activation='relu', input_shape = (28, 28, 1)))
    model.add(Conv2D(256, kernel_size = 3, activation='relu'))
    model.add(Conv2D(256, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(Dropout(0.5))

    model.add(Conv2D(128, kernel_size = 3, activation='relu'))
    model.add(Conv2D(128, kernel_size = 3, activation='relu'))
    model.add(Conv2D(128, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(Dropout(0.5))



    model.add(Conv2D(128, kernel_size = 4, activation='relu'))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    # COMPILE WITH ADAM OPTIMIZER AND CROSS ENTROPY COST
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    # model.summary()  

    return model

In [48]:
basic_model = basic_cnn_model()

models = []
for x_trn, y_trn in zip(X_train, Y_train):
  models.append(basic_model.fit(np.array(x_trn), np.array(y_trn), epochs=100, verbose=2, validation_split=0.2))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch 4/100
2/2 - 0s - loss: 2.2376 - accuracy: 0.1786 - val_loss: 2.2165 - val_accuracy: 0.4000
Epoch 5/100
2/2 - 0s - loss: 2.3199 - accuracy: 0.1250 - val_loss: 2.2447 - val_accuracy: 0.2667
Epoch 6/100
2/2 - 0s - loss: 2.1781 - accuracy: 0.2500 - val_loss: 2.2678 - val_accuracy: 0.2000
Epoch 7/100
2/2 - 0s - loss: 2.1919 - accuracy: 0.1607 - val_loss: 2.2760 - val_accuracy: 0.2000
Epoch 8/100
2/2 - 0s - loss: 2.2154 - accuracy: 0.2500 - val_loss: 2.2404 - val_accuracy: 0.2000
Epoch 9/100
2/2 - 0s - loss: 2.1380 - accuracy: 0.2143 - val_loss: 2.1793 - val_accuracy: 0.4000
Epoch 10/100
2/2 - 0s - loss: 2.0084 - accuracy: 0.3214 - val_loss: 2.1402 - val_accuracy: 0.4000
Epoch 11/100
2/2 - 0s - loss: 1.9481 - accuracy: 0.3393 - val_loss: 2.0925 - val_accuracy: 0.4667
Epoch 12/100
2/2 - 0s - loss: 1.6469 - accuracy: 0.4107 - val_loss: 2.1247 - val_accuracy: 0.2667
Epoch 13/100
2/2 - 0s - loss: 1.8010 - accuracy: 0.4286 - val_loss: 2.1698 - val_accurac

모델 완성

In [ ]:
# model.fit(X_train[0][0], Y_train[0][0], epochs = 10, verbose=2, validation_split=0.2)

NameError: ignored

In [ ]:
# annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)
# validation=0.2
# epochs = 40
# ########제너레이트 안하는 모델########
# model_list = [modela, modelb, modelc, modeld, modele, modelf, modelg, modelh, modeli, modelj, modelk, modell, modelm, modeln, modelo, modelp, modelq, modelr, models, modelt, modelu, modelv, modelw, modelx, modely, modelz]
# # x_train_list = [x_traina, x_trainb, x_trainc, x_traind, x_traine, x_trainf, x_traing, x_trainh, x_traini, x_trainj, x_traink, x_trainl, x_trainm, x_trainn, x_traino, x_trainp, x_trainq, x_trainr, x_trains, x_traint, x_trainu, x_trainv, x_trainw, x_trainx, x_trainy, x_trainz]
# # y_train_list = [y_traina, y_trainb, y_trainc, y_traind, y_traine, y_trainf, y_traing, y_trainh, y_traini, y_trainj, y_traink, y_trainl, y_trainm, y_trainn, y_traino, y_trainp, y_trainq, y_trainr, y_trains, y_traint, y_trainu, y_trainv, y_trainw, y_trainy, y_trainy, y_trainz]
# for x_train_alpha, y_train_alpha, model in tqdm(zip(X_train, Y_train, model_list)):
#   for x_train, y_train in zip(x_train_alpha, y_train_alpha):
#     model.fit(x_train, y_train, epochs=epochs, verbose=2, validation_split=validation)


ValueError: ignored

In [ ]:
basic_model = basic_cnn_model()

genmodels = []
for x_train, y_train, model in tqdm(zip(x_train_list, y_train_list, model_list)):
  X_train2, X_val2, y_train2, y_val2 = train_test_split(x_train, y_train, test_size = 0.2)
  for x_trn, y_trn in zip(X_train, Y_train):
    genmodels.append(basic_model.fit_generator(np.array(x_trn), np.array(y_trn), epochs=30, verbose=2, validation_split=0.2))

In [ ]:
BATCH_SIZE = 32
epochs = 70
########제너레이트 하는 모델########
for x_train, y_train, model in tqdm(zip(x_train_list, y_train_list, model_list)):
  X_train2, X_val2, y_train2, y_val2 = train_test_split(x_train, y_train, test_size = 0.2)

  history = model.fit_generator(
    datagen.flow(X_train2, y_train2, batch_size=BATCH_SIZE),
    epochs=epochs, 
    steps_per_epoch=X_train2.shape[0]//BATCH_SIZE,
    validation_data=(X_val2, y_val2), 
    callbacks=[annealer], 
    verbose=1
  )

In [ ]:
########테스트는 안나눠도 될듯?########
# x_test_list = [x_testa, x_testb, x_testc, x_testd, x_teste, x_testf, x_testg, x_testh, x_testi, x_testj, x_testk, x_testl, x_testm, x_testn, x_testo, x_testp, x_testq, x_testr, x_tests, x_testt, x_testu, x_testv, x_testw, x_testx, x_testy, x_testz]

model save

In [ ]:
submission = pd.read_csv('/content/drive/My Drive/데이콘 팀플/데이콘 공유/submission.csv')

sub=[]
for x, label in zip(x_test, x_test_letter):
  sub.append(np.argmax(model_list[alpha.index(label)].predict(x), axis=1))
  # submission['digit'] = np.argmax(model.predict(x_test), axis=1)
submission['digit'] = sub.toarray()

3

In [ ]:
name = 'test0906_2'#이름
submission.to_csv('/content/drive/My Drive/데이콘 팀플/{}.csv'.format(name), index=False)#경로
# model.save('/content/drive/My Drive/데이콘 팀플/{}.h5'.format(name))#경로

학습결과 시각화

Predict

In [ ]:
data_list2 = []
for i in tqdm(range(len(test))):    #data는 엑셀 데이터(즉, 원본 데이터)
  image = Image.open('/content/gdrive/My Drive/데이콘 공유/open_cv_이미지 파일/image_test_updated/image_{}.jpg'.format(i)) #위에 디렉토리에서 사진을 가져오기
  data = asarray(image)
  data_list2.append(data)
x_test = np.array(data_list2)         #저는 train 데이터를 넣어서 X_train으로 이름 지었습니다.
x_test = x_test.reshape(-1, 28, 28, 1)
x_test = x_test/255
# x_test

In [ ]:
submission = pd.read_csv('/content/gdrive/My Drive/project/Mnist cnn dacon/data/submission.csv')
submission['digit'] = np.argmax(model.predict(x_test), axis=1)
now = datetime.datetime.now().strftime('%Y-%m-%d %H_%M_%S')
submission.to_csv('DACON_08_'+ now +'.csv', index=False)

,id,digit
0,2049,6
1,2050,9
2,2051,6
3,2052,0
4,2053,3
...,...,...
20475,22524,4
20476,22525,1
20477,22526,6
20478,22527,8
